We'll start by importing the modules required to complete this project
psycopg2 - To create a connection with the database and run SQL scripts
Faker - To generate fake data for the database
Random - Used with faker to generate random data

In [1]:
import psycopg2
from faker import Faker
import random


Create database using psycopg2

In [2]:

try:
    conn = psycopg2.connect(
        host = "localhost",
        user = "yinka_admin",
        password = "yinkaadmin",
        dbname = "postgres"
    )
    conn.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT)
    cur = conn.cursor()

    cur.execute("DROP DATABASE IF EXISTS exam_db")
    conn.commit()

    cur.execute(f"CREATE DATABASE exam_db")
    conn.commit()

    print(f"Database exam_db created successfully")

except (Exception, psycopg2.Error) as error:
    print(f"Error while creating database: {error}")

finally:
    if conn:
            cur.close()
            conn.close()
            print("PostgreSQL connection is closed.")



Database exam_db created successfully
PostgreSQL connection is closed.


Connect to the database you created

In [3]:
try:
    conn = psycopg2.connect(
        host = "localhost",
        user = "yinka_admin",
        password = "yinkaadmin",
        dbname = "exam_db"
    )
    conn.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT)
    cur = conn.cursor()

    cur.execute("CREATE SCHEMA IF NOT EXISTS exam_schema")
    conn.commit()
    print("Exam schema successfully created")


except (Exception, psycopg2.Error) as error:
    print(f"Error while creating schema: {error}")

Exam schema successfully created


Create Tables for the database

In [9]:
# Create customer table
try:
    cur.execute("""
        CREATE TABLE IF NOT EXISTS customers(
                customer_id SERIAL PRIMARY KEY,
                name TEXT NOT NULL,
                email TEXT UNIQUE NOT NULL,
                phone_number TEXT,
                address JSONB)
                """
                )

    # Create product table
    cur.execute("""
                CREATE TABLE IF NOT EXISTS products(
                product_id SERIAL PRIMARY KEY,
                product_name VARCHAR(100) NOT NULL,
                category VARCHAR(100),
                price DECIMAL(10, 2) NOT NULL,
                stock_quantity INT NOT NULL 
                )
                """)

    # Create the orders table
    cur.execute("""
                CREATE TABLE IF NOT EXISTS orders(
                order_id SERIAL PRIMARY KEY,
                customer_id INT NOT NULL, 
                order_date TIMESTAMP NOT NULL,
                total_amount DECIMAL(10, 2) NOT NULL,
                FOREIGN KEY (customer_id) REFERENCES customers (customer_id)
                )
                """)

    # Create the order item table
    cur.execute("""
                CREATE TABLE IF NOT EXISTS order_items(
                order_item_id SERIAL PRIMARY KEY,
                order_id INT NOT NULL,
                product_id INT NOT NULL,
                quantity INT NOT NULL,
                price DECIMAL(10,2) NOT NULL,
                FOREIGN KEY (order_id) REFERENCES orders(order_id),
                FOREIGN KEY (product_id) REFERENCES productS(product_id)
                )
                """)
    
    conn.commit()
    print("Tables created successfully")

except (Exception, psycopg2.Error) as error:
    print(f"Error while creating table {error}")


Tables created successfully


Generate fake data and populate the tables